In [2]:
#get the mnist data 
# wget http://deeplearning.net/data/mnist/mnist.pkl.gz
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets(".", one_hot=True)

# Parameters
learning_rate = 0.001
training_epochs = 30
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 512 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    #we can add dropout layer
    # drop_out = tf.nn.dropout(layer_2, 0.75)


    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & biases
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                  "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Extracting ./train-images-idx3-ubyte.gz


Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x10e7f8eb8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x10e7f88d0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x10d203cf8>)


#2 实现两个隐藏层的Softmax
![images](../images/11/24.png)
truncated_normal:生成截断正态分布,输出从截断正态分布抽取的随机数。产生的随机数服从指定均值和标准差的正态分布，那些到均值的距离超过2倍标准差的随机数将被丢弃，然后重新抽取，直到得到足够数量的随机数为止.x的范围是$[mean-2*stddev, mean+2*stddev]$，如果mean和stddev都是默认值，那么x的范围就是-2~2

stddev是$\frac{1}{\sqrt{weights行数}}$，这是因为$y1*W2$随后的结果是一个以y1的维度为行数目，W2的列数目为列数目的向量，这样做就是以weights矩阵的函数规范化标准差，让weights矩阵中的每一列都服从0均值阶段正态分布,这样不会给输入信号添加人为偏置![images](../images/11/25.png)如果W2的某一列的均值不为0，相当于人为假如了偏置

In [ ]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import argparse
import sys
import math

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

NUM_CLASSES=10
IMAGE_SIZE=28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE

def inference(Images, hidden1_units, hidden2_units):
    with tf.name_scope('hidden1'):
        weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units], 
                                                  stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')
        hidden1 = tf.nn.relu(tf.matmul(Images, weights) + biases)
    with tf.name_scope('hidden2'):
        weights = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units], 
                                                  stddev=1.0 / math.sqrt(float(hidden1_units))),name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]), name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(tf.truncated_normal([hidden2_units, NUM_CLASSES],
                                                  stddev=1.0 / math.sqrt(float(hidden2_units))),name='weights')
        biases = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases')
        logits = tf.matmul(hidden2, weights) + biases 
    
    return logits

def loss(logits, labels): 
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, 
                                                                   logits=logits, name='xentropy') # 将logits转化为概率分布
    return tf.reduce_mean(cross_entropy, name='xentropy_mean')

def training(loss, learning_rate):
    tf.summary.scalar('loss', loss) #标量汇总
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    global_step = tf.Variable(0, name='global_step', trainable=False) # 迭代次数
    train_op = optimizer.minimize(loss=loss, global_step=global_step)
    return train_op
    
def evaluation(logits, labels):
    correct = tf.nn.in_top_k(logits, label, 1) 
    # 比较最大的一个概率跟实际比较，如果一样就成功，否则就失败；如果是2，那么就是找到最大的两个概率，只要有一个跟实际一样，那么就成功
    return tf.reduce_sum(tf.cast(correct, tf.int32))